In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras

from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.src import applications
from keras.models import Sequential, load_model
from keras.src.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.src.legacy.preprocessing import image

import cv2
import os

import warnings
warnings.filterwarnings('ignore')

2024-04-12 15:03:28.344226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 15:03:28.344331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 15:03:28.452646: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Data Augmentation on train dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Data Augmentation on test dataset
test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [3]:
cd ..

/kaggle


In [4]:
cd ..

/


In [5]:
train_generator = train_datagen.flow_from_directory(
    'kaggle/input/data/train',
    target_size=(255,255),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = test_datagen.flow_from_directory(
    'kaggle/input/data/valid',
    target_size=(255,255),
    batch_size=32,
    class_mode='categorical'
)

Found 13104 images belonging to 15 classes.
Found 300 images belonging to 15 classes.


In [6]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(255,255,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(15,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 253, 253, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    15,745,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,765,455 (60.14 MB)

 Trainable params: 15,765,455 (60.14 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
        train_generator,
        batch_size=32,
        epochs=100,
        validation_data=valid_generator,
        validation_batch_size=32,
)

Epoch 1/100
  2/410 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.1016 - loss: 8.8683   

I0000 00:00:1712934244.998406      88 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1712934245.015730      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


409/410 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2225 - loss: 3.1455

W0000 00:00:1712934712.208931      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


410/410 ━━━━━━━━━━━━━━━━━━━━ 491s 1s/step - accuracy: 0.2230 - loss: 3.1415 - val_accuracy: 0.2733 - val_loss: 2.5615
Epoch 2/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 354s 847ms/step - accuracy: 0.4730 - loss: 1.7098 - val_accuracy: 0.3800 - val_loss: 2.3139
Epoch 3/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 353s 848ms/step - accuracy: 0.5313 - loss: 1.4971 - val_accuracy: 0.3267 - val_loss: 2.1742
Epoch 4/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 354s 848ms/step - accuracy: 0.5724 - loss: 1.3478 - val_accuracy: 0.4633 - val_loss: 1.9714
Epoch 5/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 355s 853ms/step - accuracy: 0.6134 - loss: 1.2023 - val_accuracy: 0.4600 - val_loss: 2.0571
Epoch 6/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 354s 848ms/step - accuracy: 0.6394 - loss: 1.1269 - val_accuracy: 0.4600 - val_loss: 2.0850
Epoch 7/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 351s 841ms/step - accuracy: 0.6694 - loss: 1.0335 - val_accuracy: 0.5567 - val_loss: 1.6033
Epoch 8/100
410/410 ━━━━━━━━━━━━━━━━━━━━ 358s 858ms/step - accuracy: 0.6920 - loss: 

In [8]:
model.save('WheatDiseaseDetection.h5')